In [2]:
import pandas as pd
print("pandas: " + pd.__version__)
import numpy as np
print("numpy:" + np.__version__)
import matplotlib.pyplot as plt
print("matplotlib: " + np.__version__)
import seaborn as sns
print("seaborn: " + sns.__version__)

pandas: 2.3.3
numpy:2.3.4
matplotlib: 2.3.4
seaborn: 0.13.2


In [3]:
# Pfad zum Datensatz - mit Filter für unrealistische Werte 
data_path = "/home/daniel/Dokumente/cardio_train.csv"
df = pd.read_csv(data_path, sep=";")
pd.set_option('display.max_columns',None)
print("data is loaded")

data is loaded


____________
# Datenexploration und -bereinigung für Mashine Learning

**1. Daten herausfiltern, die nicht passen:** 

Es sollten Ausreißer und unrealistische Werte entfernt oder korrigiert werden  
Zum Beispiel: Eine Körpergröße von 130 cm für einen Erwachsenen ist unrealistisch und sollte überprüft werden.


**2. Probleme mit den verschiedenen Datentypen klären:**

- Datentyp int64 haben, der für numerische Daten geeignet ist, während andere Spalten möglicherweise andere Datentypen benötigen.
- Achten dass alle numerischen Spalten auch wirklich numerische Werte enthalten und keine fehlerhaften Einträge (z.B. Strings in numerischen Spalten).
- Falls cardio (die Zielvariable) als int64 kodiert ist, aber eigentlich kategorisch ist (z.B. 0 für keine Herzerkrankung, 1 für Herzerkrankung), dann ist das in Ordnung.



**Vorbereitung der Daten für das Modell**   
Um Daten in x (Eingabevariablen) und y (Zielvariable) aufzuteilen, muss entscheidieden werden, welche Variablen als Eingabe und welche als Ausgabe verwendet werden soll.  

*Beispielaufteilung:*

**x (Eingabevariablen):**

- age: Alter
- gender: Geschlecht
- height: Körpergröße
- weight: Gewicht
- ap_hi: Oberer Blutdruck (systolisch)
- ap_lo: Unterer Blutdruck (diastolisch)
- cholesterol: Cholesterinlevel
- gluc: Glukoselevel
- smoke: Raucherstatus (binär)
- alco: Alkoholkonsum (binär)
- active: Körperliche Aktivität (binär)


**y (Zielvariable):**

- cardio: Vorhandensein einer Herz-Kreislauf-Erkrankung (binär, 0 oder 1)


# Regression 
Regression ist ein übergeordneter Begriff für Machine-Learning-Modelle, die Zusammenhänge zwischen Variablen modellieren und Vorhersagen treffen.

Ziel: Eine Zielvariable (abhängige Variable) basierend auf Features (unabhängigen Variablen) vorhersagen.


**Warum logistische Regression in unseren Fall?**
Die Zielvariable "cardio" ist binär (nur 0 oder 1).

Lineare Regression wäre falsch, weil:
- Sie könnte Werte außerhalb von 0/1 vorhersagen (z. B. -0.3 oder 1.2), was unsinnig ist.
- Sie modelliert keine Wahrscheinlichkeiten.

**Logistische Regression** ist passend, weil:
- sie Wahrscheinlichkeiten zwischen 0 und 1 ausgibt (z. B. "60% Risiko für Herzerkrankung").

https://www.youtube.com/watch?v=T1nSZWAksNA

In [4]:
# alle Datentyen 
df.dtypes

id               int64
age              int64
gender           int64
height           int64
weight         float64
ap_hi            int64
ap_lo            int64
cholesterol      int64
gluc             int64
smoke            int64
alco             int64
active           int64
cardio           int64
dtype: object

# 1. Zielvariable (y) und Features (X) definieren

- **Zielvariable (y)** die man vorhersagen möchte - (0 = keine Herz-Kreislauf Erkranung, 1 = Erkrankung)
- **Features (x)** Alle anderen Spalten, die für Vorhersage genutzt werden 

In [7]:
# Zielvariable (y) und Features (X) trennen
X = df.drop(columns=["cardio", "id"])  # "id" ist irrelevant für das Modell
y = df["cardio"]

# 2. Daten aufteilen (Train/Validation/Test)

- Trainingsdaten: Zum trainieren des Modells
- (Validierungsdaten: Zur Hyperparamter-Optimierung)
- Testdaten: zur finalen Evaluation (NUR EINMALIG NUTZEN)

**sklearn.model_selection** = Untermodul der Python-Bibliothek scikit-learn (kurz: sklearn). Es enthält Funktionen zur Datenaufteilung, Modellvalidierung und mehr   
**train_test_split** Teilt Daten in Trainings- und Testsets auf.  
**stratify=y** hat dafür gesorgt, dass Train und Test dieselbe Verteilung haben (beide 50%).  

Testdaten sind heilig! - **X_test und y_test** nur einmal am Ende, um die finale Performance zu messen.

In [8]:
# Code für 80/20 Split 
from sklearn.model_selection import train_test_split

# 80% Train, 20% Test (ohne Validierung)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 20% Testdaten
    random_state=42,   # Für Reproduzierbarkeit
    stratify=y  # Wichtig, falls Klassen ungleich verteilt sind!
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Klassenverteilung Train: {y_train.mean():.2f} (Anteil Klasse 1)")
print(f"Klassenverteilung Test: {y_test.mean():.2f} (Anteil Klasse 1)")

Train: (56000, 11), Test: (14000, 11)
Klassenverteilung Train: 0.50 (Anteil Klasse 1)
Klassenverteilung Test: 0.50 (Anteil Klasse 1)


**Interpretation der Ausgabe** 
- Trainingsdaten (X_train) (56000, 11) : 56.000 Samples mit 11 Features (Spalten)
- Testdaten (X_test) (14000, 11) : 14.000 Samples (20% der Daten) mit denselben 11 Features
- **Klassenverteilung (Train)** 0.50 : 50% der Trainingsdaten gehören zur Klasse 1 (ausgeglichen!)
- **Klassenverteilung (Test)** 0.50 : 50% der Testdaten gehören zur Klasse 1 (identisch zum Training).

In [9]:
X_train.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
58394,18995,2,162,83.0,120,80,1,1,0,0,0
60371,17319,1,158,64.0,120,80,1,1,0,0,1
41399,19017,1,165,95.0,160,100,2,1,0,0,1
11468,20388,1,164,83.0,150,100,1,1,0,0,1
20650,18236,1,156,52.0,100,67,1,1,0,0,0


In [10]:
X_test.head(5)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
18682,19386,1,155,59.5,120,85,1,1,0,0,1
40992,21081,1,160,59.0,130,90,1,1,0,0,1
38068,15129,2,175,88.0,120,80,2,1,0,0,1
12096,18785,2,177,62.0,120,90,1,1,0,0,1
17791,18171,1,167,81.0,120,80,1,1,0,0,1


# 3. Modell trainieren & evaluieren

- **LogisticRegression:**  Implementiert die logistische Regression, ein lineares Klassifikationsmodell, das die Wahrscheinlichkeit einer Klasse (hier: cardio=1) vorhersagt.
- **classification_report:** Erstellt einen Bericht mit Precision, Recall, F1-Score und Accuracy für jede Klasse.
- **roc_auc_score:** 
    - Berechnet die Fläche unter der ROC-Kurve (AUC).
    - AUC = 1.0: Perfekte Trennung der Klassen.
    - AUC = 0.5: Zufällige Vorhersage (wie Münzwurf).

**model = LogisticRegression(random_state=42, max_iter=1000)**
- random_state=42 - Setzt einen Seed für die Zufallsinitialisierung, damit Ergebnisse reproduzierbar sind.
    - Die logistische Regression startet mit zufälligen Werten für die Koeffizienten ( β0​,β1​,… ).
    -  x1​,x2​,…  = Features (z. B. age, cholesterol, blood_pressure).
        -  β1​,β2​,…  = Gelernte Gewichte (z. B. wie stark age die Wahrscheinlichkeit für cardio=1 beeinflusst)
     
Die β -Werte sind gelernt und geben an, wie stark ein Feature die Zielvariable beeinflusst.  
Vor dem Training setzt das Modell die β -Werte zufällig (aber reproduzierbar mit random_state=42)  
Das Modell vergleicht seine Vorhersagen mit den echten Werten (y_train) und passt die β -Werte schrittweise an, um den Fehler zu minimieren.  
Nach dem Training (z. B. nach 1000 Iterationen) hat das Modell die β -Werte so angepasst, dass sie die Daten am besten erklären.  


- max_iter=1000 - Maximale Anzahl an Iterationen für die Optimierung

**model.fit(X_train, y_train)** == Modell trainieren
- Das Modell lernt die Gewichte ( β0​,β1​,… ) für jede der 11 Features in X_train, um die Wahrscheinlichkeit von y_train=1 zu maximieren.

**Vorhersagen generieren**
- model.predict(X_test)
- Beispiel: Falls P(y=1)=0.7  → y_pred = 1.


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Modell trainieren
model = LogisticRegression(random_state=42, max_iter=1000)  # max_iter erhöht für Konvergenz
model.fit(X_train, y_train)

# Vorhersagen auf Testdaten
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # Wahrscheinlichkeiten für ROC-AUC

# Evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("\nROC-AUC Score:", roc_auc_score(y_test, y_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.74      0.72      7004
           1       0.72      0.67      0.69      6996

    accuracy                           0.70     14000
   macro avg       0.71      0.70      0.70     14000
weighted avg       0.71      0.70      0.70     14000


ROC-AUC Score: 0.7629265450372392


/home/daniel/miniconda3/envs/my-conda-env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Was bedeuten die Metriken konkret?
***A) Precision (Genauigkeit der Vorhersage des Modells mit seinen eigenen Daten)***  
"Wie viele der vom Modell als 'krank' vorhergesagten Patienten sind tatsächlich krank?"  

**Klasse 0 (gesund): 69%**
- "Wenn das Modell vorhersagt, dass ein Patient gesund ist (cardio=0), liegt es in 69% der Fälle richtig."
- Fehlalarme (False Positives für Klasse 0):
31% der als "gesund" vorhergesagten Patienten haben tatsächlich eine Herz-Krankheit (cardio=1).
- → Problem: 31% der "gesunden" Vorhersagen sind falsch! Das könnte gefährlich sein, wenn Patienten fälschlich beruhigt werden.

**Klasse 1 (Herz-Krankheit): 72%**
- "Wenn das Modell vorhersagt, dass ein Patient krank ist (cardio=1), liegt es in 72% der Fälle richtig."  
- Fehlalarme (False Positives für Klasse 1):28% der als "krank" vorhergesagten Patienten sind tatsächlich gesund.
- Konsequenz: Unnötige weitere Tests oder Sorgen für diese Patienten.

***B) Recall (Trefferquote der echten Fälle)***  
"Wie viele der tatsächlich kranken Patienten wurden vom Modell korrekt erkannt?"  

**Klasse 0 (gesund): 74%**

- "Das Modell erkennt 74% aller tatsächlich gesunden Patienten (cardio=0) korrekt."
Verpasste gesunde Patienten (False Negatives für Klasse 0):
- 26% der gesunden Patienten werden fälschlich als krank klassifiziert.
- → Konsequenz: Unnötige Behandlungen für diese Patienten.

**Klasse 1 (Herz-Krankheit): 67%**

"Das Modell erkennt nur 67% aller tatsächlich kranken Patienten (cardio=1) korrekt."
Verpasste Krankheitsfälle (False Negatives für Klasse 1):
33% der kranken Patienten werden als gesund eingestuft!
→ Kritisch für medizinische Anwendungen!
Diese Patienten erhalten keine weitere Diagnostik oder Behandlung, obwohl sie krank sind.

***C) F1-Score (Harmonisches Mittel aus Precision und Recall)***

**Klasse 0: 72%, Klasse 1: 69%**
- Der F1-Score ist ein Ausgleich zwischen Precision und Recall.
- Interpretation: Das Modell performt etwas besser bei der Erkennung gesunder Patienten (Klasse 0) als bei kranken (Klasse 1).
- → Priorität: Den Recall für Klasse 1 verbessern (mehr kranke Patienten erkennen)!

***D) Accuracy (Gesamtgenauigkeit)***

**70%**
- "Das Modell liegt in 70% aller Fälle (14.000 Patienten) richtig."
Hier sind die Klassen fast ausgeglichen (7004 vs. 6996), also ist die Accuracy aussagekräftig.
→ 70% ist okay, aber nicht gut genug für medizinische Entscheidungen!

***E) Support (Anzahl der Samples pro Klasse)***

**Klasse 0: 7.004, Klasse 1: 6.996**
- Die Klassen sind fast perfekt balanciert (jeweils ~50%).